In [5]:
pip install cupy


     ---------------------------------------- 0.0/3.4 MB ? eta -:--:--
     ---------------------------------------- 3.4/3.4 MB 40.1 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached fastrlock-0.8.2-cp312-cp312-win_amd64.whl.metadata (9.6 kB)
Using cached fastrlock-0.8.2-cp312-cp312-win_amd64.whl (30 kB)
  Running setup.py clean for cupy
Failed to build cupy
Note: you may need to restart the kernel to use updated packages.


  error: subprocess-exited-with-error
  
  python setup.py bdist_wheel did not run successfully.
  exit code: 1
  
  [63 lines of output]
  Generating cache key from header files...
  Cache key (1610 files matching C:\Users\Laxad\AppData\Local\Temp\pip-install-5saavwai\cupy_151cfec0d07f45a0a973991c9b5224c7\cupy\_core\include\**): a33e4bcad96fb7e9f14a1faa6e6c8ddeb959e2dd
  Clearing directory: C:\Users\Laxad\AppData\Local\Temp\pip-install-5saavwai\cupy_151cfec0d07f45a0a973991c9b5224c7\cupy\.data
  Looking for NVTX: C:\Program Files\NVIDIA Corporation\Nsight Systems *\target-windows-x64\nvtx
  NVTX could not be found
  
  -------- Configuring Module: cuda --------
  a.cpp
  C:\Users\Laxad\AppData\Local\Temp\tmpd1ue9ept\a.cpp(1): fatal error C1083: Cannot open include file: 'cublas_v2.h': No such file or directory
  command 'C:\\Program Files (x86)\\Microsoft Visual Studio\\2022\\BuildTools\\VC\\Tools\\MSVC\\14.39.33519\\bin\\HostX86\\x64\\cl.exe' failed with exit code 2
  a.cpp
  C:\Users

In [13]:
"""
This example creates an optimized routing solution for a small set of customer requests.
Each request is defined as a data object containing:
    - customer name
    - pickup address and pickup time window (in minutes after midnight)
    - dropoff address and dropoff time window (in minutes after midnight)
A central waiting location (the depot) is used as the starting point for vehicles.
The code uses geopy to geocode addresses, calculates distances and travel times,
and then sets up an OR-Tools pickup and delivery routing problem.
"""

from geopy.geocoders import Nominatim  # Geocoding
from haversine import haversine, Unit  # Distance calculation between coordinates
import numpy as np  # Numerical operations
from ortools.constraint_solver import routing_enums_pb2  # OR-Tools enums
from ortools.constraint_solver import pywrapcp  # OR-Tools solver
import folium  # For map visualization
import osmnx as ox  # For street network data and routing
import networkx as nx  # For graph operations
import datetime  # For timestamping saved map

def create_data_model():
    """
    Creates the data model for the DARP problem using explicit customer requests.
    Returns:
        data (dict): Contains all necessary data (distance/time matrices, constraints, etc.).
        locations (list): List of tuples: (address, latitude, longitude) for each node.
        passenger_node_to_name (dict): Mapping from node indices (pickup/dropoff) to customer names.
    """
    # Use a generic depot address to ensure successful geocoding.
    depot_address = "Gainesville, FL"
    
    # Define a list of customer requests as data objects (dictionaries)
    # Updated time windows are wider for feasibility.
    # Times are in minutes after midnight (e.g., 540 = 9:00 AM, 720 = 12:00 PM, etc.)
    customer_requests = [
        {
            "customer_name": "Alice",
            "pickup_address": "University of Florida, Gainesville, FL",
            "pickup_time_window": (540, 720),    # 9:00 AM to 12:00 PM
            "dropoff_address": "Santa Fe College, Gainesville, FL",
            "dropoff_time_window": (660, 900)      # 11:00 AM to 3:00 PM
        },
        {
            "customer_name": "Bob",
            "pickup_address": "Shands Hospital, Gainesville, FL",
            "pickup_time_window": (600, 720),    # 10:00 AM to 12:00 PM
            "dropoff_address": "Downtown Gainesville, Gainesville, FL",
            "dropoff_time_window": (660, 900)      # 11:00 AM to 3:00 PM
        },
        {
            "customer_name": "Charlie",
            "pickup_address": "Oaks Mall, Gainesville, FL",
            "pickup_time_window": (540, 720),    # 9:00 AM to 12:00 PM
            "dropoff_address": "Ben Hill Griffin Stadium, Gainesville, FL",
            "dropoff_time_window": (660, 840)      # 11:00 AM to 2:00 PM
        },
        {
            "customer_name": "David",
            "pickup_address": "Gainesville Country Club, Gainesville, FL",
            "pickup_time_window": (600, 720),    # 10:00 AM to 12:00 PM
            "dropoff_address": "Kanapaha Botanical Gardens, Gainesville, FL",
            "dropoff_time_window": (660, 900)      # 11:00 AM to 3:00 PM
        },
        {
            "customer_name": "Eva",
            "pickup_address": "Florida Museum of Natural History, Gainesville, FL",
            "pickup_time_window": (540, 720),    # 9:00 AM to 12:00 PM
            "dropoff_address": "Devil's Millhopper Geological State Park, Gainesville, FL",
            "dropoff_time_window": (660, 840)      # 11:00 AM to 2:00 PM
        }
    ]
    
    # Combine the depot and all pickup/dropoff addresses into one list.
    # The node order will be:
    #    index 0: Depot (central waiting location)
    #    indices 1,2: Pickup and dropoff for customer 1, etc.
    addresses = [depot_address]
    for req in customer_requests:
        addresses.append(req["pickup_address"])
        addresses.append(req["dropoff_address"])
    
    # Geocode all addresses to get (latitude, longitude) for each node.
    geolocator = Nominatim(user_agent="darp_example")
    locations = []
    # Define fallback coordinates (center of Gainesville)
    fallback_lat, fallback_lon = 29.6516, -82.3248

    for address in addresses:
        location = geolocator.geocode(address)
        if location:
            locations.append((address, location.latitude, location.longitude))
        else:
            print(f"Geocoding failed for address: {address}. Using fallback coordinates.")
            locations.append((address, fallback_lat, fallback_lon))
    
    num_locations = len(locations)
    if num_locations < 2:
        raise ValueError("Not enough locations were geocoded successfully.")

    # Build the distance matrix using the haversine formula (distance in miles)
    distance_matrix = np.zeros((num_locations, num_locations))
    for i in range(num_locations):
        for j in range(num_locations):
            coord1 = (locations[i][1], locations[i][2])
            coord2 = (locations[j][1], locations[j][2])
            distance_matrix[i][j] = haversine(coord1, coord2, unit=Unit.MILES)

    # Estimate the travel time matrix assuming an average speed (mph)
    average_speed = 30  # miles per hour
    time_matrix = (distance_matrix / average_speed) * 60  # travel times in minutes

    # Build the data dictionary required by OR-Tools.
    data = {}
    # Scale distances and times to integers (scaling factor: 1000)
    data['distance_matrix'] = (distance_matrix * 1000).astype(int).tolist()
    data['time_matrix'] = (time_matrix * 1000).astype(int).tolist()
    data['num_vehicles'] = 4
    data['vehicle_capacities'] = [6] * data['num_vehicles']
    data['depot'] = 0  # Depot is the central waiting location

    # Define pickup and delivery pairs, demands, and build a mapping from node indices to customer names.
    data['pickups_deliveries'] = []
    passenger_node_to_name = {}
    data['demands'] = [0] * num_locations  # 0 demand for the depot
    for i, req in enumerate(customer_requests):
        pickup_index = 1 + 2 * i
        dropoff_index = 1 + 2 * i + 1
        data['pickups_deliveries'].append([pickup_index, dropoff_index])
        passenger_node_to_name[pickup_index] = req["customer_name"]
        passenger_node_to_name[dropoff_index] = req["customer_name"]
        data['demands'][pickup_index] = 1   # Pickup increases load
        data['demands'][dropoff_index] = -1  # Dropoff decreases load

    # Define time windows for each location.
    # Depot: 24-hour window (0 to 1440 minutes)
    data['time_windows'] = [(0, 1440)]
    for req in customer_requests:
        data['time_windows'].append(req["pickup_time_window"])
        data['time_windows'].append(req["dropoff_time_window"])

    return data, locations, passenger_node_to_name

def main():
    """
    Main function to solve the DARP and visualize the routes.
    """
    # Create the data model from customer request objects.
    data, locations, passenger_node_to_name = create_data_model()

    # Create the routing index manager.
    manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']),
                                           data['num_vehicles'],
                                           data['depot'])

    # Create the routing model.
    routing = pywrapcp.RoutingModel(manager)

    # ******* Define Cost and Constraint Callbacks *******

    # Distance callback: returns the travel cost between two nodes.
    def distance_callback(from_index, to_index):
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data['distance_matrix'][from_node][to_node]

    transit_callback_index = routing.RegisterTransitCallback(distance_callback)
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

    # Demand callback: returns the demand at each node.
    def demand_callback(from_index):
        from_node = manager.IndexToNode(from_index)
        return data['demands'][from_node]

    demand_callback_index = routing.RegisterUnaryTransitCallback(demand_callback)
    routing.AddDimensionWithVehicleCapacity(
        demand_callback_index,
        0,  # no slack
        data['vehicle_capacities'],  # vehicle capacities
        True,  # start cumul at zero
        'Capacity')

    # Time callback: returns travel time between nodes.
    def time_callback(from_index, to_index):
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data['time_matrix'][from_node][to_node]

    time_callback_index = routing.RegisterTransitCallback(time_callback)
    routing.AddDimension(
        time_callback_index,
        30 * 1000,  # allow waiting time (scaled in same units)
        24 * 60 * 60 * 1000,  # maximum route duration (24 hours)
        False,  # do not force start cumul to zero
        'Time')
    time_dimension = routing.GetDimensionOrDie('Time')

    # Set time window constraints for each location.
    for location_idx, time_window in enumerate(data['time_windows']):
        index = manager.NodeToIndex(location_idx)
        # Multiply time windows by 60 * 1000 to match scaling (minutes -> scaled units)
        time_dimension.CumulVar(index).SetRange(time_window[0] * 60 * 1000,
                                                time_window[1] * 60 * 1000)

    # Add pickup and delivery constraints.
    for request in data['pickups_deliveries']:
        pickup_index = manager.NodeToIndex(request[0])
        dropoff_index = manager.NodeToIndex(request[1])
        routing.AddPickupAndDelivery(pickup_index, dropoff_index)
        routing.solver().Add(
            routing.VehicleVar(pickup_index) == routing.VehicleVar(dropoff_index))
        routing.solver().Add(
            time_dimension.CumulVar(pickup_index) <= time_dimension.CumulVar(dropoff_index))

    # ******* Configure the Solver Search Parameters *******
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC
    search_parameters.local_search_metaheuristic = (
        routing_enums_pb2.LocalSearchMetaheuristic.GUIDED_LOCAL_SEARCH)
    search_parameters.time_limit.FromSeconds(120)

    # ******* Solve the Problem *******
    solution = routing.SolveWithParameters(search_parameters)

    if solution:
        # Extract routes for each vehicle.
        routes = []
        for vehicle_id in range(data['num_vehicles']):
            index = routing.Start(vehicle_id)
            route = []
            while not routing.IsEnd(index):
                node_index = manager.IndexToNode(index)
                route.append(node_index)
                index = solution.Value(routing.NextVar(index))
            routes.append(route)

        # Visualize the routes on a map.
        m = visualize_routes(routes, data, locations, passenger_node_to_name)
        timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
        filename = f'routes_map_{timestamp}.html'
        m.save(filename)
        print(f'Map has been saved to {filename}')
    else:
        print('No solution found!')

def visualize_routes(routes, data, locations, passenger_node_to_name):
    """
    Visualizes the routes on a Folium map.
    Args:
        routes (list): List of routes for each vehicle.
        data (dict): Data model containing distances, demands, and time windows.
        locations (list): List of locations with (address, latitude, longitude).
        passenger_node_to_name (dict): Mapping of node indices to customer names.
    Returns:
        m (folium.Map): Map with the plotted routes.
    """
    # Center the map around Gainesville.
    m = folium.Map(location=[29.6516, -82.3248], zoom_start=12)
    
    # Build a street network graph for routing.
    G = ox.graph_from_place('Alachua County, Florida, USA', network_type='drive')
    G = G.to_undirected()

    # Define colors for different vehicles.
    colors = ['red', 'blue', 'green', 'purple', 'orange']
    
    # Map each location to its nearest node in the street network.
    node_ids = []
    for loc in locations:
        lat, lon = loc[1], loc[2]
        try:
            node = ox.distance.nearest_nodes(G, lon, lat)
            node_ids.append(node)
        except Exception as e:
            print(f"Error mapping location {loc[0]}: {e}")
            node_ids.append(None)
    
    # Plot each vehicle's route.
    for vehicle_id, route in enumerate(routes):
        color = colors[vehicle_id % len(colors)]
        path = []
        valid_route = True
        for i in range(len(route) - 1):
            orig_node = node_ids[route[i]]
            dest_node = node_ids[route[i + 1]]
            if orig_node is None or dest_node is None:
                print(f"Skipping segment from {route[i]} to {route[i+1]} due to missing mapping.")
                valid_route = False
                break
            try:
                route_segment = nx.shortest_path(G, orig_node, dest_node, weight='length')
                path.extend(route_segment)
            except nx.NetworkXNoPath:
                print(f"No path found between nodes {orig_node} and {dest_node}")
                valid_route = False
                break
        if not valid_route or not path:
            print(f"Skipping route for vehicle {vehicle_id} due to issues.")
            continue

        # Convert node IDs to coordinates.
        path_coords = [(G.nodes[node]['y'], G.nodes[node]['x']) for node in path]
        folium.PolyLine(path_coords, color=color, weight=2.5, opacity=1).add_to(m)
        
        # Add markers for each node on the route.
        for node_index in route:
            lat, lon = locations[node_index][1], locations[node_index][2]
            coord = (lat, lon)
            if data['demands'][node_index] == 1:
                # Mark pickup locations
                name = passenger_node_to_name.get(node_index, 'Unknown')
                folium.Marker(coord, icon=folium.Icon(color=color, icon='plus'),
                              popup=f'Pickup: {name}').add_to(m)
            elif data['demands'][node_index] == -1:
                # Mark dropoff locations
                name = passenger_node_to_name.get(node_index, 'Unknown')
                folium.Marker(coord, icon=folium.Icon(color=color, icon='minus'),
                              popup=f'Dropoff: {name}').add_to(m)
            else:
                # Depot marker
                folium.Marker(coord, icon=folium.Icon(color='black', icon='home'),
                              popup='Depot').add_to(m)
    return m

if __name__ == '__main__':
    main()


KeyboardInterrupt: 